In [11]:
import pandas as pd
from clean_images import CleanImages
from clean_tabular import CleanData
import os
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import re
from PIL import Image
import multiprocessing
from torchvision.transforms import Normalize, ToPILImage, ToTensor
from torch.nn import Module
from torch import nn
import torch.optim as optim
from pathlib import Path
from pytorch_scratch_classification import Dataset
from torchvision import models, datasets
import copy

In [4]:
model = models.resnet50(pretrained=True)
model
for param in model.parameters():
    param.requires_grad = False

In [5]:
model.fc = nn.Linear(in_features=2048, out_features=13, bias=True)

In [6]:
train_transformer = transforms.Compose([transforms.RandomRotation(40), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
train_dataset = Dataset(transformer=train_transformer, X='image', img_size=224, is_test=False)

images
                                     id                            product_id  \
0  912bb259-3ad9-457b-9db1-ce1da9016057  5f5f57d7-778f-4336-bb10-b43863418c8c   
1  b166d305-b852-4bdd-83f4-465b20da94fa  5f5f57d7-778f-4336-bb10-b43863418c8c   
2  68f5a29d-0075-4d60-81c1-ab684a82e50c  c2c8949f-3cde-4651-a234-4a4a1b2a9ad4   
3  f6a309d7-d247-446a-9b5e-aceefdd4334d  c2c8949f-3cde-4651-a234-4a4a1b2a9ad4   
4  2c2b3a6f-15b3-4289-937a-15482d9f5781  8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf   

                                         bucket_link         image_ref  \
0  https://aicore-product-images.s3.amazonaws.com...  koQAAOSwhZRiE5dD   
1  https://aicore-product-images.s3.amazonaws.com...  9vgAAOSw8sRiE5dE   
2  https://aicore-product-images.s3.amazonaws.com...  FlwAAOSwCkpiFNlV   
3  https://aicore-product-images.s3.amazonaws.com...  4gIAAOSw4ORiFNlZ   
4  https://aicore-product-images.s3.amazonaws.com...  sHoAAOSwQoRiD7vf   

  create_time  
0  2022-02-26  
1  2022-02-26  
2  2022-02-26

In [7]:
test_transformer = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_dataset = Dataset(transformer=train_transformer, X='image', img_size=224, is_test=True)

images
                                     id                            product_id  \
0  912bb259-3ad9-457b-9db1-ce1da9016057  5f5f57d7-778f-4336-bb10-b43863418c8c   
1  b166d305-b852-4bdd-83f4-465b20da94fa  5f5f57d7-778f-4336-bb10-b43863418c8c   
2  68f5a29d-0075-4d60-81c1-ab684a82e50c  c2c8949f-3cde-4651-a234-4a4a1b2a9ad4   
3  f6a309d7-d247-446a-9b5e-aceefdd4334d  c2c8949f-3cde-4651-a234-4a4a1b2a9ad4   
4  2c2b3a6f-15b3-4289-937a-15482d9f5781  8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf   

                                         bucket_link         image_ref  \
0  https://aicore-product-images.s3.amazonaws.com...  koQAAOSwhZRiE5dD   
1  https://aicore-product-images.s3.amazonaws.com...  9vgAAOSw8sRiE5dE   
2  https://aicore-product-images.s3.amazonaws.com...  FlwAAOSwCkpiFNlV   
3  https://aicore-product-images.s3.amazonaws.com...  4gIAAOSw4ORiFNlZ   
4  https://aicore-product-images.s3.amazonaws.com...  sHoAAOSwQoRiD7vf   

  create_time  
0  2022-02-26  
1  2022-02-26  
2  2022-02-26

In [8]:
batch_size = 20
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)
data_loader_dict = {'train': train_loader, 'eval': test_loader}

In [10]:
optimizer =  optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [12]:
model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[ 1.3335e-02,  1.4664e-02, -1.5351e-02,  ..., -4.0896e-02,
                         -4.3034e-02, -7.0755e-02],
                        [ 4.1205e-03,  5.8477e-03,  1.4948e-02,  ...,  2.2060e-03,
                         -2.0912e-02, -3.8517e-02],
                        [ 2.2331e-02,  2.3595e-02,  1.6120e-02,  ...,  1.0281e-01,
                          6.2641e-02,  5.1977e-02],
                        ...,
                        [-9.0349e-04,  2.7767e-02, -1.0105e-02,  ..., -1.2722e-01,
                         -7.6604e-02,  7.8453e-03],
                        [ 3.5894e-03,  4.8006e-02,  6.2051e-02,  ...,  2.4267e-02,
                         -3.3662e-02, -1.5709e-02],
                        [-8.0029e-02, -3.2238e-02, -1.7808e-02,  ...,  3.5359e-02,
                          2.2439e-02,  1.7077e-03]],
              
                       [[-1.8452e-02,  1.1415e-02,  2.3850e-02,  ...,  5.3736e-02,
                          4.4022

In [ ]:
train_size = 10000
test_size = 2424
dataset_size = {'train': train_size, 'test': test_size}
'Model training and testing function'
def train_model(model=model, optimizer=optimizer, loss_type = criterion, num_epochs = 20):
    best_model_weights = copy.deepcopy(model.state_dict()) #May be changed at end of each "for phase block"
    best_accuracy = 0 # May be changed at end of each "for phase block"

    for epoch in range(num_epochs):
        for phase in ['train', 'eval']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0
            running_corrects = 0

            for inputs, labels in data_loader_dict[phase]:
                optimizer.zero_grad() # Gradients reset to zero at beginning of both training and evaluation phase

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward() #Calculates gradients
                        optimizer.step()
                
                running_loss = running_loss + loss.item()*inputs.size(0)
                running_corrects = running_corrects + torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects.double() / dataset_size[phase]

            if phase == 'eval' and epoch_acc > best_accuracy:
                best_accuracy = epoch_acc
                best_model_weights = copy.deepcopy(model.state_dict())
    model.load_state_dict(best_model_weights)
    return model
